In [15]:
from collaborative import (
    recommend_for_user as collaborative_recommendation, 
    train_knn_model as collaborative_train_knn_model, 
    split_train_test_data as collaborative_split_train_test_data, 
    knn_model_train as collaborative_knn_model_train, 
    train_data as collaborative_train_data, 
    test_data as collaborative_test_data,
    asin_mapping as collaborative_asin_mapping
)
from ContentBase import (
    recommend_similar_items as content_based_recommendation, 
    kmeans as content_kmeans_model, 
    data as content_data, 
    X_train as content_X_train, 
    X_test as content_X_test, 
    train_test_split as content_train_test_split
)
from sklearn.model_selection import cross_val_score
import numpy as np
import random

def ensemble_recommendation(user_id, user_item_index, collaborative_train_data, collaborative_test_data, knn_model_train, content_train_data, content_X_test, kmeans_model, content_data, asin_mapping, title_mapping, N=10):
    # Generate collaborative recommendations
    collaborative_recs = collaborative_recommendation(user_id, collaborative_train_data, collaborative_test_data, knn_model_train, asin_mapping, N)
    
    # Generate content-based recommendations
    content_based_recs = content_based_recommendation(user_item_index, kmeans_model, content_data, content_X_test)

    # Convert recommendations to lists
    if isinstance(collaborative_recs, str):
        collaborative_recs = [collaborative_recs]
    if isinstance(content_based_recs, str):
        content_based_recs = [content_based_recs]

    # Initialize lists to store recommendations
    ensemble_recs = []

    # Map collaborative recommendations to content-based titles
    for rec in collaborative_recs:
        # Get the title from the title_mapping
        title = title_mapping.get(rec, "Unknown Title")
        # Append the title along with the recommendation
        ensemble_recs.append((title, rec, 'collaborative'))

    # Add content-based recommendations to ensemble recommendations
    for rec in content_based_recs:
        ensemble_recs.append((rec, 'content-based'))

    # Sort ensemble recommendations by score
    ensemble_recs = sorted(ensemble_recs, key=lambda x: x[1], reverse=True)[:N]

    # Extract item IDs from ensemble recommendations
    ensemble_recs = [item[0] for item in ensemble_recs]

    return '\n'.join(ensemble_recs)




In [16]:

asin_title_df = content_data[['asin', 'title']]

# Convert DataFrame to dictionary
title_mapping = dict(zip(asin_title_df['asin'], asin_title_df['title']))

# Usage example
user_id = random.randint(1, 2088619)
user_item_index = random.randint(0, len(content_X_test) - 1)
knn_model_train = collaborative_knn_model_train
kmeans_model = content_kmeans_model
kmeans_model.fit(content_X_train)

# Call the ensemble recommendation function
ensemble_recommendations = ensemble_recommendation(user_id, user_item_index, collaborative_train_data, collaborative_test_data, knn_model_train, content_X_train, content_X_test, kmeans_model, content_data, collaborative_asin_mapping, title_mapping)

# Print the ensemble recommendations
print("\nEnsemble Recommendations:")
for asin in ensemble_recommendations.split('\n'):
    print(asin)  # Assuming ASINs are printed directly


No ratings found for the user. Recommending popular items.
Here are the top 10 most popular items: ['1608838137', 'B003EYVXV4', 'B00FZM8Z7I', 'B004NSUXHU', 'B0038M2APA', 'B001KVZ6HK', 'B0001VL0K2', 'B003WE9C5U', 'B0000AQS0F', 'B00AF6B22E']

Ensemble Recommendations:
Everyday Italian (with Giada de Laurentiis), Volume 1 (3 Pack): Italian Classics, Parties, Holidays
Why Don't They Just Quit? DVD Roundtable Discussion: What Families and Friends need to Know About Addiction and Recovery
Understanding Seizures and Epilepsy DVD
Live in Houston [VHS]
Rise and Swine (Good Eats Vol. 7)
Barefoot Contessa (with Ina Garten), Entertaining With Ina Vol. 2 (3 Pack): Brunch 'n' Lunch, Picnic Parties, Summer Entertaining
Peace Child [VHS]
Worship with Don Moen [VHS]
The Life of Sister Faustina: The Apostle of Divine Mercy [VHS]
Witnesses of Jehovah [VHS]
unknown
unknown
unknown
unknown
unknown
unknown
unknown
The Lord of the Rings: The Motion Picture Trilogy (Widescreen Edition)
Firefly: The Complete S